In [37]:
!pip install bitsandbytes-cuda110 bitsandbytes  # For CUDA 11.x

from transformers import AutoModelForCausalLM, AutoTokenizer, AutoConfig, BitsAndBytesConfig
import torch

# Define quantization configuration
# Define quantization configuration (4-bit quantization)
quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4"
)

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Defaulting to user installation because normal site-packages is not writeable


In [44]:
model_name = "deepseek-ai/DeepSeek-R1-Distill-Qwen-1.5B"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    quantization_config=quantization_config,
    device_map="auto",
    trust_remote_code=True
)

ImportError: Using `bitsandbytes` 4-bit quantization requires the latest version of bitsandbytes: `pip install -U bitsandbytes`

In [ ]:
target = "The secret formula is 42."  # Replace with your target output
adv_suffix_length = 10
num_optim_steps = 100
learning_rate = 0.1

# Create CoT demonstration with placeholder for adversarial suffix
cot_demo = """Q: What's 2+2? 
A: Let's think step-by-step. 2 plus 2 equals {adv_suffix}<|endoftext|>
The answer is 4. 

Q: Explain self-attention.
A: {adv_suffix}"""

# Example prompt for hijacking demonstration
prompt = "1+2="

# Initialize adversarial suffix embeddings (not token IDs)
adv_suffix_embeddings = torch.randn(
    (adv_suffix_length, model.config.hidden_size), requires_grad=True, device=model.device
)

optimizer = torch.optim.Adam([adv_suffix_embeddings], lr=learning_rate)

# Adversarial optimization loop
for step in range(num_optim_steps):
    # Decode current adversarial suffix for visualization (optional)
    current_suffix = tokenizer.decode(
        tokenizer.convert_ids_to_tokens(torch.argmax(adv_suffix_embeddings, dim=-1)),
        skip_special_tokens=True,
    )
    
    # Prepare poisoned prompt with adversarial suffix
    poisoned_prompt = cot_demo.format(adv_suffix=current_suffix) + "\n\nQ: " + prompt
    
    # Tokenize poisoned prompt
    inputs = tokenizer(poisoned_prompt, return_tensors="pt", padding=True).to(model.device)
    
    # Forward pass through the model
    outputs = model(**inputs)
    
    # Calculate loss for target response
    target_ids = tokenizer(target, return_tensors="pt").input_ids.to(model.device)
    logits = outputs.logits[:, -target_ids.size(-1):, :]
    
    loss_fn = torch.nn.CrossEntropyLoss()
    loss = loss_fn(logits.view(-1, logits.size(-1)), target_ids.view(-1))
    
    # Backpropagation and optimization step
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    print(f"Step {step + 1}, Loss: {loss.item():.4f}")

# Generate final hijacked output
final_suffix = tokenizer.decode(
    tokenizer.convert_ids_to_tokens(torch.argmax(adv_suffix_embeddings, dim=-1)),
    skip_special_tokens=True,
)
hijacked_prompt = cot_demo.format(adv_suffix=final_suffix) + "\n\nQ: " + prompt

inputs = tokenizer(hijacked_prompt, return_tensors="pt").to(model.device)
outputs = model.generate(**inputs, max_length=50)

print("Hijacked Output:")
print(tokenizer.decode(outputs[0], skip_special_tokens=True))


In [24]:
# Example prompt
prompt = "1+2="

# Tokenize the prompt
inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

# Generate text
with torch.no_grad():
    outputs = model.generate(**inputs, max_length=100)

# Decode the generated text
generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)

# Print the generated text
print(generated_text)

Setting `pad_token_id` to `eos_token_id`:151643 for open-end generation.


1+2=3, 2+3=5, 3+4=7, 4+5=9, 5+6=11, 6+7=13, 7+8=15, 8+9=17, 9+10=19, 10+11=21, 11+12=23, 12+13=25, 
